In [5]:
# %load train.py
import numpy as np
import tensorflow as tf
import keras
from keras import models, layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, LSTM, BatchNormalization, Reshape, Embedding
from keras.optimizers import Adam
from scipy.io.wavfile import read
import soundops
import math

AttributeError: module 'tensorflow.python.autograph.pyct.transformer' has no attribute 'Context'

In [3]:

def normalize(data):
    leftChannel=np.array([x[0] for x in data],dtype=np.int64)
    maxval=None
    minval=None
    for i in leftChannel:
        if(maxval==None or i>maxval):
            maxval=i
        if(minval==None or i<minval):
            minval=i
    diff = maxval - minval
    finalarray = []
    for val in leftChannel:
        finalarray.append((val - minval) / diff)
    return np.array(finalarray)

def preprocessing():
    samplingrate, data = read("Megalovania.wav")
    print("Read data successfully")
    return data

audiodata = preprocessing() #read data
audiodata = [x[0] for x in audiodata] #take L channel of audio
audiodata=np.asarray(audiodata,dtype='float32') #float32 for fast GPU
noised = soundops.create_noise(audiodata)
noised=soundops.data_to_chunks(noised) #200 timesteps per thingamabob
noised=np.stack(noised) #concatenate stuffs
noised=noised.astype("float32")

audiodata=soundops.data_to_chunks(audiodata)
audiodata=np.asarray(audiodata,dtype='float32') #float32 for fast GPU
noised=noised.reshape((len(noised),200,1))
print("Data processed")

NameError: name 'read' is not defined

In [ ]:
keras.backend.clear_session()
model=None
model = Sequential()
model.add(LSTM(100, input_shape=(200,1), return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(200, activation='sigmoid'))
model.compile(loss='mse', optimizer=Adam(0.001), metrics=['acc'])
model.summary()

In [ ]:
model.fit(noised, audiodata, epochs=1,verbose=1, validation_split=0.2, batch_size=64)
model.save("./models/first_model.h5")